# Analysis of Mass Spec data

Adapted from `main.py`.

Run the original script either using something like [Spyder](https://www.spyder-ide.org/), or from the command line with command `python main.py <directory> output_name=<output_name> plot=<plot_diagnostic> stable_col=<stable_col>`, where all arguments are optional.
- `directory` : path
  - Location (relative or absolute) to the location in which data is stored. Optional, default: .\data
- `output_name` : str
  - Name of the csv containing summary data. Optional, default: output.csv
- `plot` : bool
  - True/False switch for plotting summary data. Optional, default: False
- `stable_col` : str
  - Column heading used to determine which data is considered stable. Optional, default: 11B

For the example data provided for which this script was developed, run
  ```python main.py .\data output_name=output.csv plot=True stable_col=11B```

In [1]:
#%% Imports

from massspecanalysis import *

import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.interpolate import interp1d
import sys

## Main body

Original script takes command-line arguments. Redefine them here as parameters.

In [2]:
dirname = os.path.join(".", "example_data")  # Takes care of OS path differences.
output_name = "output.csv"                   # Should include filetype. To save as an Excel spreadsheet, change the `to_csv()` method in the
                                             #   last cell to `to_excel()`.
plot_diagnostic = True                       # Save figures.
stable_col = "11B"                           # Needs to be identical to column name in *.exp files.

Initialise a folder to store data. Keeps things tidy.

In [3]:
files = os.listdir(dirname)
if "output" not in files:
    os.mkdir(os.path.join(dirname, "output"))

To include regression of the blank signal, we need to pass through the data twice: once to collect the data for regression, and once to do the correction. More complicated regressions may depend on signal at a later time, so we cannot rely on only doing one pass.

In [4]:
#%% Regression step
gas_signals = pd.DataFrame([])
for filename in files:
    # Check filetype, make sure that we get output data only. Note *.exp is just a text file with a different filetype.
    if filename[-4:] == ".exp":
        # Read in the data from this file. Alternative formats of data (e.g. from different machines) require a new read function.
        info, data = read_exp(os.path.join(dirname, filename), skipfooter=13, index_col=0, sep='\t', engine='python', usecols=range(8))
        mmnt_cols = data.columns[data.columns != "Time"]
        # Work out when the data is stable. Use 11B signal to do it.
        start_cycle, end_cycle = stable_cycles(data["Time"], data[stable_col])
        # Else define a custom point to start measuring the gas signal.
        # Use case tends to be ~500 cycles long, get the last ~50.
        end_cycle = 450
        # Work out what the blank gas signal is for all data cols.
        gas_signal, first_blank_cycle = blank_signal(data.loc[end_cycle:, "Time"], data.loc[end_cycle:, mmnt_cols], factor=3)
        # Store it for regression later.
        gas_signals = pd.concat([gas_signals, pd.DataFrame(gas_signal, columns=[filename[:-4]]).transpose()])                

Second pass: process the data and correct using the data stored in `gas_signals`. Note that signals are regressed in the `column` loop in this cell - to change the kind of regression, change the `kind` argument in `interp1d` [(documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d). Simple examples are `kind="next"`, which subtracts the gas value of the upcoming experiment (current behaviour), and `kind="linear"` performs a linear interpolation between the previous and next points.

(N.B. could move this up to inputs rather than written in the code here - expectation that this would be set and not changed, so leave it here for now.)

In [6]:
#%% Second pass: process the stable data.
processed_data = pd.DataFrame([])
for filename in files:
    # Check filetype
    if filename[-4:] == ".exp":
        # Read in the data.
        info, data = read_exp(os.path.join(dirname, filename), skipfooter=13, index_col=0, sep='\t', engine='python', usecols=range(8))
        mmnt_cols = data.columns[data.columns != "Time"]
        # Work out when sample is stable using 11B. Returns indices within which the sample is considered stable.
        start_cycle, end_cycle = stable_cycles(data["Time"], data[stable_col])
        stable_data = data.loc[start_cycle:end_cycle, data.columns != "Time"]

        # Regress the blank signals in time.
        stable_corrected = stable_data.copy()
        mmnt_time = [timestamp.timestamp() for timestamp in data.loc[start_cycle:end_cycle, "Time"].to_list()]
        gas_time  = [timestamp.timestamp() for timestamp in gas_signals.loc[:, "Time"].to_list()]
        for column in mmnt_cols:
            gas_vals = gas_signals.loc[:, column+" Mean"].to_list()
            # interp1d creates a function whose input is the query points. Change "kind" for different type of interpolation,
            # options at https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d
            # Note that kind="next" subtracts the constant value coming up (this is the current behaviour)
            # And kind="linear" does a linear interpolation between previous and next points.
            func = interp1d(gas_time, gas_vals, kind="next", bounds_error=False, fill_value=(gas_vals[0], gas_vals[-1]))
            stable_corrected.loc[:, column] -= func(mmnt_time)
            
        # Recalculate ratios
        stable_corrected.loc[:, "11B/10B (1)"] = stable_corrected.loc[:, "11B"] / stable_corrected.loc[:, "10B"]
        stable_corrected.loc[:, "11B/10.066 (2)"] = stable_corrected.loc[:, "11B"] / stable_corrected.loc[:, "10.066"]

        # Combine stable_data and stable_corrected to remove any outliers.
        # Note that we only need one copy of Time column, so remove the one
        # in stable_corrected.
        corrected_cols = stable_corrected.columns + " Corrected"
        stable_corrected.columns = corrected_cols
        stable_data = pd.concat([stable_data, stable_corrected[mmnt_cols+" Corrected"]], axis=1)

        # Remove outliers
        outlier_cols = ["11B/10B (1)", "11B/10B (1) Corrected"]
        stable_data = remove_outliers(stable_data, outlier_cols)

        #%% Do the analysis.
        mean   = stable_data.mean(axis=0)
        std    = stable_data.std(axis=0)
        cycles = len(stable_data)

        # Do the plotting
        if plot_diagnostic:
            # 11B vs cycle number: Showing stability.
            title = "{} Stable Data".format(filename[:-4])
            fig = plt.figure(figsize=(10, 5), dpi=100)
            ax = fig.add_subplot(111)
            ax.scatter(data.index, data[stable_col], 4, label="All data")
            ax.scatter(stable_data.index, stable_data[stable_col], 4, label="Stable data")
            ax.scatter(data.loc[first_blank_cycle:].index, data.loc[first_blank_cycle:, stable_col], 4, label="Gas signal")
            ax.set_xlabel("Cycle Number")
            ax.set_ylabel(f"{stable_col} Voltage")
            ax.set_title(title)
            ax.legend(loc=0)
            ax.grid()
            fig.savefig(os.path.join(dirname, "output", title+".png"))
            plt.close()

            # 11/10 B vs cycle number: Showing correction.
            title = "{} Correction".format(filename[:-4])
            fig = plt.figure(figsize=(10, 5), dpi=100)
            ax = fig.add_subplot(111)
            ax.scatter(stable_data.index, stable_data["11B/10B (1)"], 4, label="Uncorrected")
            ax.scatter(stable_data.index, stable_data["11B/10B (1) Corrected"], 4, label="Corrected")
            ax.set_xlabel("Cycle Number")
            ax.set_ylabel("11/10 B Voltage")
            ax.set_title(title)
            ax.legend(loc=0)
            ax.grid()
            fig.savefig(os.path.join(dirname, "output", title+".png"))
            plt.close()

        output_data = rearrange_means(mean, std, pd.Series(cycles, index=['Cycles']))

        #%% Store the data.
        processed_data = pd.concat([processed_data, pd.DataFrame(output_data, columns=[filename[:-4]]).transpose()])

In [7]:
#%% Save the output
processed_data.to_csv(os.path.join(dirname, "output", output_name))